In [21]:
from ethereum import tester
import os

contract_code = """
event Notice(x:str)

alice = 0x{alice}
bob = 0x{bob}

def load_money():
   # Must pay before time #1
   if block.number > 1: 
      log(type=Notice, text("load_money: called after block 1"))
      return(0)

   if msg.value != 10*(10**21):
      log(type=Notice, text("load_money: wrong amount"))
      return(0)

   if msg.sender == alice:
      log(type=Notice, text("load_money: Alice OK!"))

   if msg.sender == bob:
      log(type=Notice, text("load_money: Bob OK!"))

   # Mark user as having paid
   self.storage[msg.sender] = 1 

def cash_out():
   # Critical block is #2
   # Must wait for block #3 to cash out
   if block.number < 3:
      log(type=Notice, text("cash_out: called before block 3"))
      return(0)

   if not self.storage[alice]:
      log(type=Notice, text("cash_out: Alice didn't pay!"))
      send(bob, self.balance)
      return(0)

   if not self.storage[bob]:
      log(type=Notice, text("cash_out: Bob didn't pay!"))
      send(alice, self.balance)
      return(0)

   block2 = block.prevhash(block.number - 2)
   if block2 % 2 == 0:
      log(type=Notice, text("cash_out: Alice won!"))
      send(alice, self.balance)

   else:
      log(type=Notice, text("cash_out: Bob won!"))
      send(bob, self.balance)

"""

s = tester.state()

# Use default addresses for Alice and Bob
alice = tester.a0
bob = tester.a1

print 'Initial balances:'
print 'Alice: %.2f' % (float(s.block.get_balance(alice)) / 10E21)
print '  Bob: %.2f' % (float(s.block.get_balance(bob)) / 10E21)

# Create the contract
full_code = contract_code.format(alice=alice.encode('hex'),
                                 bob=bob.encode('hex'))
contract = s.abi_contract(full_code)

# Both parties deposit money
# Comment out this line, it will shift the timeline too fast...
#s.mine(3)
contract.load_money(value=int(10E21), sender=tester.k0) # Alice
contract.load_money(value=int(10E21), sender=tester.k1) # Bob

# Mine some blocks
s.block.extra_data = os.urandom(20) # Add actual randomness
# If you mine 1, cashes are depositted
# If you mine 2, the lottery result is out; but not cashed
# If you mine 3, result is cashed out
s.mine(3) 

# Run the cash_out 
contract.cash_out()

print 'Final balances:'
print 'Alice: %.2f' % (float(s.block.get_balance(alice)) / 10E21)
print '  Bob: %.2f' % (float(s.block.get_balance(bob)) / 10E21)


Initial balances:
Alice: 100.00
  Bob: 100.00
{u'x': 'load_money: Alice OK!', '_event_type': 'Notice'}
{u'x': 'load_money: Bob OK!', '_event_type': 'Notice'}
{u'x': 'cash_out: called before block 3', '_event_type': 'Notice'}
Final balances:
Alice: 99.00
  Bob: 99.00


In [15]:
s.blocks[0].transactions.__dict__

{'db': <ethereum.db._EphemDB at 0x7fd135c82b10>,
 'death_row_timeout': 5000,
 'journal': [],
 'nodes_for_death_row': [],
 'root_node': ['\x94\x8c\x00DP\xed\x9a9\xffxp\x1a\xf2\xc6\x90-\xd6^0C3\x96\xcf\r\xad\xe7\x9c0\xcd\x85 %',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '3:\x15yH\x841\xc9e*\x84\x03^\xa7\x8f5\x16\xa7\xc0\x90\x83\xb5.\x1fzv\xae\x17\xa7\x11kY',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  ''],
 'transient': False}

In [24]:
s.mine(3)
contract.load_money(value=int(10E21), sender=tester.k0) # Alice
contract.load_money(value=int(10E21), sender=tester.k1) # Bob

# Mine some blocks
s.block.extra_data = os.urandom(20) # Add actual randomness
s.mine(3)

# Run the cash_out 
contract.cash_out()
print 'Final balances:'
print 'Alice: %.2f' % (float(s.block.get_balance(alice)) / 10E21)
print '  Bob: %.2f' % (float(s.block.get_balance(bob)) / 10E21)


{u'x': 'load_money: called after block 1', '_event_type': 'Notice'}
{u'x': 'load_money: called after block 1', '_event_type': 'Notice'}
{u'x': "cash_out: Alice didn't pay!", '_event_type': 'Notice'}
Final balances:
Alice: 79.06
  Bob: 121.00


In [29]:
print tester.state()

<ethereum.tester.state instance at 0x7fdce238d440>


In [39]:
alice.encode('hex')

'82a978b3f5962a5b0957d9ee9eef472ee55b42f1'

In [34]:
!ifconfig

eth0      Link encap:Ethernet  HWaddr fa:16:3e:74:fb:0c  
          inet addr:192.168.11.132  Bcast:192.168.11.255  Mask:255.255.255.0
          inet6 addr: fe80::f816:3eff:fe74:fb0c/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1454  Metric:1
          RX packets:48093 errors:0 dropped:0 overruns:0 frame:0
          TX packets:9169 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:11910663 (11.9 MB)  TX bytes:5067787 (5.0 MB)

lo        Link encap:Local Loopback  
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
          UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:2285 errors:0 dropped:0 overruns:0 frame:0
          TX packets:2285 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:0 
          RX bytes:4359824 (4.3 MB)  TX bytes:4359824 (4.3 MB)



In [2]:
help(os.urandom)

Help on built-in function urandom in module posix:

urandom(...)
    urandom(n) -> str
    
    Return n random bytes suitable for cryptographic use.



In [13]:
tester.k1.encode('hex')   # This is the public key

'c89efdaa54c0f20c7adf612882df0950f5a951637e0307cdcb4c672f298b8bc6'

In [14]:
tester.a1.encode('hex')   # This is the private key

'7d577a597b2742b498cb5cf0c26cdcd726d39e6e'

no Python documentation found for '\xc8\x9e\xfd\xaaT\xc0\xf2\x0cz\xdfa(\x82\xdf\tP\xf5\xa9Qc~\x03\x07\xcd\xcbLg/)\x8b\x8b\xc6'

